In [1]:
from pathlib import Path

import eelbrain
import mne
from scipy.io import loadmat
import numpy as np



# Data locations
DATA_ROOT = Path("~").expanduser() / 'Data' / 'cocoha'
DATA_PREPROC = DATA_ROOT / "data_preprocessed"



In [2]:

for file in DATA_PREPROC.glob("*.mat"):

    SUBJECT = file.stem.split("_")[0]

    dst_dir = DATA_ROOT / 'eeg' / SUBJECT
    dst_dir.mkdir(exist_ok=True, parents=True)
    dst = dst_dir / f'{SUBJECT}_raw.fif'
    if dst.exists():
        continue


    print("Saving ~", dst)

    mat = loadmat(file, squeeze_me=True, struct_as_record=False)
    data = mat['data']

    # --- collect all trials ---
    trials = []

    for trial in data.eeg:  # 60 trials
        eeg = trial[:, :64]        # remove extra channels
        trials.append(eeg)

    # concatenate along time
    eeg_concat = np.concatenate(trials, axis=0)  # (total_time, 64)

    # transpose for MNE (channels, time)
    eeg_concat = eeg_concat.T

    # --- create MNE Raw ---
    ch_names = data.dim.chan.eeg[0][:64].tolist()

    info = mne.create_info(
        ch_names=ch_names,
        sfreq=64,
        ch_types='eeg'
    )

    raw = mne.io.RawArray(eeg_concat, info)
    raw.set_montage('biosemi64')

    # --- save ---
    raw.save(dst)


Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S5/S5_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S15/S15_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S11/S11_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S1/S1_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S4/S4_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S14/S14_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S10/S10_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S17/S17_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S7/S7_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S18/S18_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S3/S3_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S13/S13_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S8/S8_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S16/S16_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S6/S6_raw.fif
Saving ~ /Users/sylvestereley/Data/cocoha/eeg/S2/S2_raw.fif
Saving ~ /Users/sylveste

In [3]:
# Test see that you get an NDVar with 64 sensor and concatenated time dimension 3200 * 60 = 192000 time points
raw = mne.io.read_raw(DATA_ROOT / 'eeg' / "S1" / "S1_raw.fif")
eeg = eelbrain.load.mne.raw_ndvar(raw)
eeg


<NDVar 'S1_raw.fif': 64 sensor, 192000 time>